# Td Idf

In [1]:
import numpy as np
import pandas as pd
import config
import jupyter_service
import sys
import matplotlib.pyplot as plt
from dbcontext import Context
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
pd.options.display.float_format = '{:.2f}'.format
print(sys.version, np.__version__, pd.__version__, config.version)
context = Context()
context.create(echo=False)
service = jupyter_service.JupyterService(context)
print(config.connection_string)

3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)] 2.0.2 2.2.3 0.1.0.Alpha
postgresql://postgres:sa@localhost:5432/yt1


In [2]:
question_nb=12
mode="tdidf"

In [3]:
df = service.get_stats(mode)
avg=df.iloc[0].average
median=df.iloc[0].median_
percentile=df.iloc[0].percentile
df

,average,median_,quartile,decile,percentile
0,51.24,13.00,5.00,3.00,2.00


Empathy Positive Score

In [22]:
empathy_category="empathy"
df = service.get_scores(mode, question_nb, empathy_category, positive=True, denominator_thresold=0, numerator_thresold=avg, debug=False)
print(f"{empathy_category} positive score")
df.head(10)

empathy positive score


,id,topic,score_positive_form,score_negative_form,score
0,9790,love,130.56,1.87,45.45
1,9773,planet,64.52,0.97,32.80
2,9654,peaceful,62.79,0.94,32.32
3,9089,happy,215.32,5.97,30.87
4,9796,happiness,270.68,7.80,30.77
5,9487,health,124.04,3.38,28.32
6,9058,able,66.47,1.46,27.05
7,9062,life,146.86,5.35,23.14
8,9785,environment,66.55,1.90,22.97
9,9674,live,110.75,3.92,22.53


Empathy Negative Score

In [5]:
df = service.get_scores(mode, question_nb, empathy_category, positive=False, denominator_thresold=0, numerator_thresold=avg)
print(f"{empathy_category} negative score")
df.head(10)


pd negative score


,id,topic,score_negative_form,score_positive_form,score
0,9449,stability,57.92,34.29,1.64
1,9712,success,87.07,76.19,1.13
2,9487,health,72.99,64.04,1.12
3,9764,family,64.65,57.56,1.10
4,9593,successful,64.26,63.68,0.99
5,9796,happiness,135.46,136.51,0.99
6,9800,good,76.14,80.96,0.93
7,9073,peace,174.78,191.74,0.91
8,9790,love,53.38,59.78,0.88
9,9062,life,73.74,86.18,0.85


Low Noise

In [21]:
empathy_category="empathy"
df = service.get_scores(mode, question_nb, empathy_category, positive=False, denominator_thresold=0, numerator_thresold=0)
df=df[df.score > avg] 
print(f"Low noise {empathy_category} negative score")
df.head(10)

Low noise empathy negative score


,id,topic,score_negative_form,score_positive_form,score


In [19]:
df = service.get_scores(mode, question_nb, empathy_category, positive=True, denominator_thresold=0, numerator_thresold=0)
df=df[df.score > avg] 
print(f"Low noise {empathy_category} positive score")
df.head(10)

Low noise empathy positive score


,id,topic,score_positive_form,score_negative_form,score
0,13171,comfort,24.50,NaN,245.05
1,9070,fulfilled,23.20,NaN,231.95
2,13354,unity,21.32,NaN,213.22
3,9229,contribute,21.31,NaN,213.15
4,13356,something,20.40,NaN,204.05
5,13012,home,20.25,NaN,202.46
6,12998,loved,18.28,NaN,182.79
7,9634,helping,16.10,NaN,161.00
8,11268,along,15.70,NaN,156.96
9,9472,start,14.72,NaN,147.24
